In [ ]:
from EmQuantAPI import *
from datetime import timedelta, datetime
import time as _time
from datetime import *
import traceback
import numpy as np
import pandas as pd
from pandas.plotting import table
import plotly_express as px
import plotly.graph_objects as go
#import matplotlib as mpl
#import matplotlib.pyplot as plt
#import xlwings as xw
#import xlrd
#from pandas import DataFrame
#from pylab import mpl

In [ ]:
def mainCallback(quantdata):
    """
    mainCallback 是主回调函数，可捕捉如下错误
    在start函数第三个参数位传入，该函数只有一个为c.EmQuantData类型的参数quantdata
    :param quantdata:c.EmQuantData
    :return:
    """
    print ("mainCallback",str(quantdata))
    #登录掉线或者 登陆数达到上线（即登录被踢下线） 这时所有的服务都会停止
    if str(quantdata.ErrorCode) == "10001011" or str(quantdata.ErrorCode) == "10001009":
        print ("Your account is disconnect. You can force login automatically here if you need.")
    #行情登录验证失败（每次连接行情服务器时需要登录验证）或者行情流量验证失败时，会取消所有订阅，用户需根据具体情况处理
    elif str(quantdata.ErrorCode) == "10001021" or str(quantdata.ErrorCode) == "10001022":
        print ("Your all csq subscribe have stopped.")
    #行情服务器断线自动重连连续6次失败（1分钟左右）不过重连尝试还会继续进行直到成功为止，遇到这种情况需要确认两边的网络状况
    elif str(quantdata.ErrorCode) == "10002009":
        print ("Your all csq subscribe have stopped, reconnect 6 times fail.")
    # 行情订阅遇到一些错误(这些错误会导致重连，错误原因通过日志输出，统一转换成EQERR_QUOTE_RECONNECT在这里通知)，正自动重连并重新订阅,可以做个监控
    elif str(quantdata.ErrorCode) == "10002012":
        print ("csq subscribe break on some error, reconnect and request automatically.")
    # 资讯服务器断线自动重连连续6次失败（1分钟左右）不过重连尝试还会继续进行直到成功为止，遇到这种情况需要确认两边的网络状况
    elif str(quantdata.ErrorCode) == "10002014":
        print ("Your all cnq subscribe have stopped, reconnect 6 times fail.")
    # 资讯订阅遇到一些错误(这些错误会导致重连，错误原因通过日志输出，统一转换成EQERR_INFO_RECONNECT在这里通知)，正自动重连并重新订阅,可以做个监控
    elif str(quantdata.ErrorCode) == "10002013":
        print ("cnq subscribe break on some error, reconnect and request automatically.")
    # 资讯登录验证失败（每次连接资讯服务器时需要登录验证）或者资讯流量验证失败时，会取消所有订阅，用户需根据具体情况处理
    elif str(quantdata.ErrorCode) == "10001024" or str(quantdata.ErrorCode) == "10001025":
        print("Your all cnq subscribe have stopped.")
    else:
        pass

def startCallback(message):
    print("[EmQuantAPI Python]", message)
    return 1
def csqCallback(quantdata):
    """
    csqCallback 是csq订阅时提供的回调函数模板。该函数只有一个为c.EmQuantData类型的参数quantdata
    :param quantdata:c.EmQuantData
    :return:
    """
    print ("csqCallback,", str(quantdata))

def cstCallBack(quantdata):
    '''
    cstCallBack 是日内跳价服务提供的回调函数模板
    '''
    for i in range(0, len(quantdata.Codes)):
        length = len(quantdata.Dates)
        for it in quantdata.Data.keys():
            print(it)
            for k in range(0, length):
                for j in range(0, len(quantdata.Indicators)):
                    print(quantdata.Data[it][j * length + k], " ",end = "")
                print()
def cnqCallback(quantdata):
    """
    cnqCallback 是cnq订阅时提供的回调函数模板。该函数只有一个为c.EmQuantData类型的参数quantdata
    :param quantdata:c.EmQuantData
    :return:
    """
    # print ("cnqCallback,", str(quantdata))
    print("cnqCallback,")
    for code in quantdata.Data:
        total = len(quantdata.Data[code])
        for k in range(0, len(quantdata.Data[code])):
            print(quantdata.Data[code][k])

try:
    #调用登录函数（激活后使用，不需要用户名密码）
    loginResult = c.start("ForceLogin=1", '', mainCallback)
    if(loginResult.ErrorCode != 0):
        print("login in fail")
        exit()

    # 市场特征
    date = datetime.today().strftime("%Y-%m-%d")
    # 指数涨幅
    zs0=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGE","N=-0,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")    
    zs5=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-5,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    zs10=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-10,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    zs20=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-20,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    zs60=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-60,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    zs120=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-120,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    zs250=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-250,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    # 指数 （日）主力净流入资金
    date = (datetime.today() + timedelta(days = -0)).strftime("%Y-%m-%d")
    zszj0=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    date = (datetime.today() + timedelta(days = -1)).strftime("%Y-%m-%d")
    zszj1=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    date = (datetime.today() + timedelta(days = -2)).strftime("%Y-%m-%d")
    zszj2=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    # 指数走势
    #date = datetime.today().strftime("%Y-%m-%d")
    zszs=c.csd("000985.CSI,000300.SH,000905.SH,000852.SH","CLOSE","2005-01-01",""+date+"","period=3,adjustflag=3,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    print(zszs)
    #zszs3=c.csd("000985.CSI,000300.SH,000905.SH,000852.SH","CLOSE","2020-01-01",""+date+"","period=1,adjustflag=3,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    # 指数估值
    zspb=c.csd("000985.CSI,000300.SH,000905.SH,000852.SH","PBMRQ","2013-01-01",""+date+"","DelType=1,period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    zspb3=c.csd("000985.CSI,000300.SH,000905.SH,000852.SH","PBMRQ","2020-01-01",""+date+"","DelType=1,period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    zspbclose=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","SHORTNAME,PBMRQ","TradeDate="+date+",DelType=1,Rowindex=none,Ispandas=1")

    # 风格涨幅
    date = datetime.today().strftime("%Y-%m-%d")
    fg0=c.css("399373.SZ,399375.SZ,399377.SZ,399372.SZ,399374.SZ,399376.SZ","NAME,DIFFERRANGE","N=0,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    fg5=c.css("399373.SZ,399375.SZ,399377.SZ,399372.SZ,399374.SZ,399376.SZ","NAME,DIFFERRANGEN","N=-5,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    fg10=c.css("399373.SZ,399375.SZ,399377.SZ,399372.SZ,399374.SZ,399376.SZ","NAME,DIFFERRANGEN","N=-10,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    fg20=c.css("399373.SZ,399375.SZ,399377.SZ,399372.SZ,399374.SZ,399376.SZ","NAME,DIFFERRANGEN","N=-20,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    fg60=c.css("399373.SZ,399375.SZ,399377.SZ,399372.SZ,399374.SZ,399376.SZ","NAME,DIFFERRANGEN","N=-60,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    fg120=c.css("399373.SZ,399375.SZ,399377.SZ,399372.SZ,399374.SZ,399376.SZ","NAME,DIFFERRANGEN","N=-120,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    fg250=c.css("399373.SZ,399375.SZ,399377.SZ,399372.SZ,399374.SZ,399376.SZ","NAME,DIFFERRANGEN","N=-250,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    # 风格资金
    date = (datetime.today() + timedelta(days = -0)).strftime("%Y-%m-%d")
    fgzj0=c.css("399373.SZ,399375.SZ,399377.SZ,399372.SZ,399374.SZ,399376.SZ","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    date = (datetime.today() + timedelta(days = -1)).strftime("%Y-%m-%d")
    fgzj1=c.css("399373.SZ,399375.SZ,399377.SZ,399372.SZ,399374.SZ,399376.SZ","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    date = (datetime.today() + timedelta(days = -2)).strftime("%Y-%m-%d")
    fgzj2=c.css("399373.SZ,399375.SZ,399377.SZ,399372.SZ,399374.SZ,399376.SZ","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    # 风格走势
    date = datetime.today().strftime("%Y-%m-%d")
    fgzs=c.csd("399373.SZ,399377.SZ,399372.SZ,399376.SZ","CLOSE","2003-01-01",""+date+"","period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    fgzs3=c.csd("399373.SZ,399377.SZ,399372.SZ,399376.SZ","CLOSE","2020-01-01",""+date+"","period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    # 风格估值
    zspb=c.csd("399373.SZ,399377.SZ,399372.SZ,399376.SZ","PBMRQ","2013-01-01",""+date+"","DelType=1,period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    zspb3=c.csd("399373.SZ,399377.SZ,399372.SZ,399376.SZ","PBMRQ","2020-01-01",""+date+"","DelType=1,period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    zspbclose=c.css("399373.SZ,399377.SZ,399372.SZ,399376.SZ","SHORTNAME,PBMRQ","TradeDate="+date+",DelType=1,Rowindex=none,Ispandas=1")

    # 行业风格涨幅
    date = datetime.today().strftime("%Y-%m-%d")
    hyfg0=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","NAME,DIFFERRANGE","N=0,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hyfg5=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","NAME,DIFFERRANGEN","N=-5,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hyfg10=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","NAME,DIFFERRANGEN","N=-10,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hyfg20=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","NAME,DIFFERRANGEN","N=-20,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hyfg60=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","NAME,DIFFERRANGEN","N=-60,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hyfg120=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","NAME,DIFFERRANGEN","N=-120,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hyfg250=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","NAME,DIFFERRANGEN","N=-250,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    # 行业风格资金
    date = (datetime.today() + timedelta(days = -0)).strftime("%Y-%m-%d")
    hyfgzj0=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    date = (datetime.today() + timedelta(days = -1)).strftime("%Y-%m-%d")
    hyfgzj1=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    date = (datetime.today() + timedelta(days = -2)).strftime("%Y-%m-%d")
    hyfgzj2=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    # 行业风格走势
    date = datetime.today().strftime("%Y-%m-%d")
    hyfgzs =c.csd("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","CLOSE","2003-01-01",""+date+"","period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    hyfgzs3 =c.csd("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","CLOSE","2020-01-01",""+date+"","period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    # 行业风格估值
    zspb=c.csd("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","PBMRQ","2013-01-01",""+date+"","DelType=1,period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    zspb3=c.csd("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","PBMRQ","2020-01-01",""+date+"","DelType=1,period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    zspbclose=c.css("CI005917.CI,CI005918.CI,CI005919.CI,CI005920.CI,CI005921.CI","SHORTNAME,PBMRQ","TradeDate="+date+",DelType=1,Rowindex=none,Ispandas=1")

    # 行业涨幅
    date = datetime.today().strftime("%Y-%m-%d")
    hy0=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","NAME,DIFFERRANGE","N=-0,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hy0=hy0.sort_values(by="DIFFERRANGE",ascending=False)
    hy5=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","NAME,DIFFERRANGEN","N=-5,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hy5=hy5.sort_values(by="DIFFERRANGEN",ascending=False)
    hy10=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","NAME,DIFFERRANGEN","N=-10,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hy10=hy10.sort_values(by="DIFFERRANGEN",ascending=False)
    hy20=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","NAME,DIFFERRANGEN","N=-20,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hy20=hy20.sort_values(by="DIFFERRANGEN",ascending=False)
    hy60=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","NAME,DIFFERRANGEN","N=-60,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hy60=hy60.sort_values(by="DIFFERRANGEN",ascending=False)
    hy120=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","NAME,DIFFERRANGEN","N=-120,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hy120=hy120.sort_values(by="DIFFERRANGEN",ascending=False)
    hy250=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","NAME,DIFFERRANGEN","N=-250,TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hy250=hy250.sort_values(by="DIFFERRANGEN",ascending=False)
    # 行业资金
    date = (datetime.today() + timedelta(days = -0)).strftime("%Y-%m-%d")
    hyzj0=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    hyzj = hyzj0.sort_values(by="NETINFLOW",ascending=False)
    date = (datetime.today() + timedelta(days = -1)).strftime("%Y-%m-%d")
    hyzj1=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    date = (datetime.today() + timedelta(days = -2)).strftime("%Y-%m-%d")
    hyzj2=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Rowindex=none,Ispandas=1")
    # 行业走势
    date = datetime.today().strftime("%Y-%m-%d")
    hyzs =c.csd("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","CLOSE","2003-01-01",""+date+"","period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    hyzs3 =c.csd("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","CLOSE","2020-01-01",""+date+"","period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    # 行业市净率PB(MRQ) 
    hypb=c.csd("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","PBMRQ","2013-01-01",""+date+"","DelType=1,period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    hypb3=c.csd("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","PBMRQ","2020-01-01",""+date+"","DelType=1,period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    hypbclose=c.css("801010.SWI,801030.SWI,801040.SWI,801050.SWI,801080.SWI,801110.SWI,801120.SWI,801130.SWI,801140.SWI,801150.SWI,801160.SWI,801170.SWI,801180.SWI,801200.SWI,801210.SWI,801230.SWI,801710.SWI,801720.SWI,801730.SWI,801740.SWI,801750.SWI,801760.SWI,801770.SWI,801780.SWI,801790.SWI,801880.SWI,801890.SWI,801950.SWI,801960.SWI,801970.SWI,801980.SWI","SHORTNAME,PBMRQ","TradeDate="+date+",DelType=1,Rowindex=none,Ispandas=1")

    # 投资策略
    date = datetime.today().strftime("%Y-%m-%d")
    # 指数PB
    agpb=c.csd("000002.SH","PBLYR","2003-01-01",""+date+"","DelType=1,period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    agpb3=c.csd("000002.SH","PBLYR","2020-01-01",""+date+"","DelType=1,period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")

    # 指数PE
    agpe=c.csd("000002.SH","PETTM","2003-01-01",""+date+"","DelType=1,period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    agpe3=c.csd("000002.SH","PETTM","2020-01-01",""+date+"","DelType=1,period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")

    # 小盘大盘比
    xp=c.csd("399316.SZ","CLOSE","2003-01-01",""+date+"","period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    dp=c.csd("399314.SZ","CLOSE","2003-01-01",""+date+"","period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    xp3=c.csd("399316.SZ","CLOSE","2020-01-01",""+date+"","period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    dp3=c.csd("399314.SZ","CLOSE","2020-01-01",""+date+"","period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")

    # 成长价值比
    cz=c.csd("399370.SZ","CLOSE","2003-01-01",""+date+"","period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    jz=c.csd("399371.SZ","CLOSE","2003-01-01",""+date+"","period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    cz3=c.csd("399370.SZ","CLOSE","2020-01-01",""+date+"","period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")
    jz3=c.csd("399371.SZ","CLOSE","2020-01-01",""+date+"","period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Rowindex=none,Ispandas=1")

#退出
    data = logoutResult = c.stop()
except Exception as ee:
    print("error >>>",ee)
    traceback.print_exc()
else:
    print("demo end")

In [ ]:

def zszfb():
    # 指数涨幅表
    # 数据合并
    zshb=pd.concat([zs0,zs5,zs10,zs20,zs60,zs120,zs250],names=None,axis=1,ignore_index=True) 
    # 删除无用列
    zshb.drop(zshb.columns[[0,1,4,5,6,8,9,10,12,13,14,16,17,18,20,21,22,24,25,26]],axis = 1,inplace = True)
    # 变更列名
    zshb.columns=['指数', '当日涨幅', '累计5日涨幅', '累计10日涨幅', '累计20日涨幅', '累计60日涨幅', '累计120日涨幅', '累计250日涨幅']
    # 删除特定字符
    zshb.指数 = zshb.指数.str.replace('指数', '')
    # 设置小数位
    zshb.当日涨幅=zshb.当日涨幅.map(lambda x:('%.1f')%x)
    zshb.累计5日涨幅=zshb.累计5日涨幅.map(lambda x:('%.0f')%x)
    zshb.累计10日涨幅=zshb.累计10日涨幅.map(lambda x:('%.0f')%x)
    zshb.累计20日涨幅=zshb.累计20日涨幅.map(lambda x:('%.0f')%x)
    zshb.累计60日涨幅=zshb.累计60日涨幅.map(lambda x:('%.0f')%x)
    zshb.累计120日涨幅=zshb.累计120日涨幅.map(lambda x:('%.0f')%x)
    zshb.累计250日涨幅=zshb.累计250日涨幅.map(lambda x:('%.0f')%x)
    fig = go.Figure(
        data=[go.Table(
            header=dict(values=list(zshb.columns),  # 表头取值是data列属性
                        fill_color='paleturquoise',align=['center','center'],font_size=17,
            height=60),  # 填充色和文本位置
                    
            cells=dict(values=[zshb.指数,zshb.当日涨幅,zshb.累计5日涨幅,zshb.累计10日涨幅,zshb.累计20日涨幅,zshb.累计60日涨幅,zshb.累计120日涨幅,zshb.累计250日涨幅],  # 单元格的取值就是每个列属性的Series取值
                    fill_color='lavender',align=['center','right'],font_size=22,
            height=60)
        )]
    )
    fig.update_layout(width=1200,height=600)
    fig.show()
    return

def zszf0():
    # 指数涨幅
    zs0['NAME'] = zs0['NAME'].str.replace('指数', '')
    fig = px.bar(zs0,x='NAME',y='DIFFERRANGE',text='DIFFERRANGE')
    fig.update_traces(texttemplate='%{text:.1f}',textposition='inside',marker=dict(color=np.where(np.array(zs0['DIFFERRANGE'])>0,'red','limegreen'))) 
    fig.update_layout(width=1200,height=600,title={'text': "当日涨幅%",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=45,font_size=35,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    fig.show()
    return

def zszf5():
    zs5['NAME'] = zs5['NAME'].str.replace('指数', '')
    fig = px.bar(zs5,x='NAME',y='DIFFERRANGEN',text='DIFFERRANGEN')
    fig.update_traces(texttemplate='%{text:.0f}',textposition='inside',marker=dict(color=np.where(np.array(zs5['DIFFERRANGEN'])>0,'red','limegreen'))) 
    fig.update_layout(width=1200,height=600,title={'text': "累计5日涨幅%",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=45,font_size=35,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    fig.show()
    return

def zszf10():
    zs10['NAME'] = zs10['NAME'].str.replace('指数', '')
    fig = px.bar(zs10,x='NAME',y='DIFFERRANGEN',text='DIFFERRANGEN')
    fig.update_traces(texttemplate='%{text:.0f}',textposition='inside',marker=dict(color=np.where(np.array(zs10['DIFFERRANGEN'])>0,'red','limegreen'))) 
    fig.update_layout(width=1200,height=600,title={'text': "累计10日涨幅%",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=45,font_size=35,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    fig.show()
    return

def zszf20():
    zs20['NAME'] = zs20['NAME'].str.replace('指数', '')
    fig = px.bar(zs20,x='NAME',y='DIFFERRANGEN',text='DIFFERRANGEN')
    fig.update_traces(texttemplate='%{text:.0f}',textposition='inside',marker=dict(color=np.where(np.array(zs20['DIFFERRANGEN'])>0,'red','limegreen'))) 
    fig.update_layout(width=1200,height=600,title={'text': "累计20日涨幅%",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=45,font_size=35,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    fig.show()
    return

def zszf60():
    zs60['NAME'] = zs60['NAME'].str.replace('指数', '')
    fig = px.bar(zs60,x='NAME',y='DIFFERRANGEN',text='DIFFERRANGEN')
    fig.update_traces(texttemplate='%{text:.0f}',textposition='inside',marker=dict(color=np.where(np.array(zs60['DIFFERRANGEN'])>0,'red','limegreen'))) 
    fig.update_layout(width=1200,height=600,title={'text': "累计60日涨幅%",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=45,font_size=35,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    fig.show()
    return

def zszf120():
    zs120['NAME'] = zs120['NAME'].str.replace('指数', '')
    fig = px.bar(zs120,x='NAME',y='DIFFERRANGEN',text='DIFFERRANGEN')
    fig.update_traces(texttemplate='%{text:.0f}',textposition='inside',marker=dict(color=np.where(np.array(zs120['DIFFERRANGEN'])>0,'red','limegreen'))) 
    fig.update_layout(width=1200,height=600,title={'text': "累计120日涨幅%",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=45,font_size=35,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    fig.show()
    return

def zszf250():
    zs250['NAME'] = zs250['NAME'].str.replace('指数', '')
    fig = px.bar(zs250,x='NAME',y='DIFFERRANGEN',text='DIFFERRANGEN')
    fig.update_traces(texttemplate='%{text:.0f}',textposition='inside',marker=dict(color=np.where(np.array(zs250['DIFFERRANGEN'])>0,'red','limegreen'))) 
    fig.update_layout(width=1200,height=600,title={'text': "累计250日涨幅%",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=45,font_size=35,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    fig.show()
    return

def zszj0():
    # 指数资金
    zszj0['NAME'] = zszj0['NAME'].str.replace('指数', '')
    zszj0['NETINFLOW'] = zszj0['NETINFLOW']/100000000
    fig = px.bar(zszj0,x='NAME',y='NETINFLOW',text='NETINFLOW')
    fig.update_traces(texttemplate='%{text:.0f}',textposition='inside',marker=dict(color=np.where(np.array(zszj0['NETINFLOW'])>0,'red','limegreen'))) 
    fig.update_layout(width=1200,height=600,title={'text': "当日指数资金（亿元）",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=45,font_size=35,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    fig.show()
    return

def zszj3():    
    # 资金数据合并
    zszj3=pd.concat([zszj0,zszj1,zszj2],names=None,axis=1,ignore_index=True)
    zszj3['zszj3']=zszj3[3]+zszj3[7]+zszj3[11]
    zszj3['zszj3'] = zszj3['zszj3']/100000000
    # 删除无用列
    zszj3.drop(zszj3.columns[[0,1,4,5,6,8,9,10]],axis = 1,inplace = True)
    # 变更列名
    zszj3.columns=['指数', 'zszj0', 'zszj1', 'zszj2', 'zszj3']
    # 删除特定字符
    zszj3['指数'] = zszj3['指数'].str.replace('指数', '')

    fig = px.bar(zszj3,x='指数',y='zszj3',text='zszj3')
    fig.update_traces(texttemplate='%{text:.0f}',textposition='inside',marker=dict(color=np.where(np.array(zszj3['zszj3'])>0,'red','limegreen'))) 
    fig.update_layout(width=1200,height=600,title={'text': "3日指数资金（亿元）",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=45,font_size=35,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    fig.show()
    return

def zszs20(): 
    # 指数走势
    # 变更列名
    zszs.columns=['指数','日期','收盘价']
    zszs['指数'] = zszs['指数'].str.replace('000985.CSI','中证全指')
    zszs['指数'] = zszs['指数'].str.replace('000300.SH','沪深300')
    zszs['指数'] = zszs['指数'].str.replace('000905.SH','中证500')
    zszs['指数'] = zszs['指数'].str.replace('000852.SH','中证1000')
    fig = px.line(zszs,x='日期', y='收盘价',color='指数')
    fig.update_layout(xaxis = dict(tickmode='linear',tick0 = 1,dtick = 24,),width=1200,height=600,title={'text': "指数走势20年",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                    title_font_size=35,font_size=15,title_font_color='red',xaxis_tickangle=-45,xaxis_title=None,yaxis_title=None)
    fig.update_yaxes(type='log')
    fig.show()
    return

def zszs3(): 
    # 3年指数走势
    # 变更列名
    zszs3.columns=['指数','日期','收盘价']
    zszs3['指数'] = zszs3['指数'].str.replace('000985.CSI','中证全指')
    zszs3['指数'] = zszs3['指数'].str.replace('000300.SH','沪深300')
    zszs3['指数'] = zszs3['指数'].str.replace('000905.SH','中证500')
    zszs3['指数'] = zszs3['指数'].str.replace('000852.SH','中证1000')
    fig = px.line(zszs3,x='日期', y='收盘价',color='指数')
    fig.update_layout(xaxis = dict(tickmode='linear',tick0 = 1,dtick = 120,),width=1200,height=600,title={'text': "指数走势3年",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                    title_font_size=35,font_size=15,title_font_color='red',xaxis_tickangle=-45,xaxis_title=None,yaxis_title=None)
    fig.update_yaxes(type='log')
    fig.show()
    return

def zspbfw20(): 
    # 指数估值
    # 分位图20年
    # 变更特定字符
    zspb['CODES'] = zspb['CODES'].str.replace('000985.CSI', '中证全指')
    zspb['CODES'] = zspb['CODES'].str.replace('000300.SH', '沪深300')
    zspb['CODES'] = zspb['CODES'].str.replace('000905.SH', '中证500')
    zspb['CODES'] = zspb['CODES'].str.replace('000852.SH', '中证1000')
    # 保留1位小数
    zspbclose['PBMRQ'] = zspbclose['PBMRQ'].apply(lambda x:format(x,'.1f'))
    y0 = zspbclose.PBMRQ[0]
    y1 = zspbclose.PBMRQ[1]
    y2 = zspbclose.PBMRQ[2]
    y3 = zspbclose.PBMRQ[3]
    fig = px.violin(zspb,x="CODES",y="PBMRQ",color="CODES",box=True,points='all')
    fig.update_layout(width=1200,height=600,title={'text': "指数PB分位20年",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                        title_font_size=35,font_size=15,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    #fig.update_yaxes(type='log')
    fig.add_annotation(x=0,y=y0,text="现值{}".format(y0),ax=-55,ay=0)
    fig.add_annotation(x=1,y=y1,text="现值{}".format(y1),ax=-55,ay=0)
    fig.add_annotation(x=2,y=y2,text="现值{}".format(y2),ax=-55,ay=0)
    fig.add_annotation(x=3,y=y3,text="现值{}".format(y3),ax=-55,ay=0)
    fig.show()
    return

def zspbzs20():
    # 走势图20年
    fig = px.line(zspb,x='DATES', y='PBMRQ', color='CODES')
    fig.update_layout(xaxis = dict(tickmode='linear',tick0 = 1,dtick = 24,),width=1200,height=600,title={'text': "指数PB走势20年",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                    title_font_size=35,font_size=15,title_font_color='red',xaxis_tickangle=-45,xaxis_title=None,yaxis_title=None)
    fig.update_yaxes(type='log')
    fig.show()
    return

def zspbfw3(): 
    # 分位图3年
    # 变更特定字符
    zspb3['CODES'] = zspb3['CODES'].str.replace('000985.CSI', '中证全指')
    zspb3['CODES'] = zspb3['CODES'].str.replace('000300.SH', '沪深300')
    zspb3['CODES'] = zspb3['CODES'].str.replace('000905.SH', '中证500')
    zspb3['CODES'] = zspb3['CODES'].str.replace('000852.SH', '中证1000')
    y0 = zspbclose.PBMRQ[0]
    y1 = zspbclose.PBMRQ[1]
    y2 = zspbclose.PBMRQ[2]
    y3 = zspbclose.PBMRQ[3]
    fig = px.violin(zspb3,x="CODES",y="PBMRQ",color="CODES",box=True,points='all')
    fig.update_layout(width=1200,height=600,title={'text': "指数PB分位3年",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                        title_font_size=35,font_size=15,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
    #fig.update_yaxes(type='log')
    fig.add_annotation(x=0,y=y0,text="现值{}".format(y0),ax=-55,ay=0)
    fig.add_annotation(x=1,y=y1,text="现值{}".format(y1),ax=-55,ay=0)
    fig.add_annotation(x=2,y=y2,text="现值{}".format(y2),ax=-55,ay=0)
    fig.add_annotation(x=3,y=y3,text="现值{}".format(y3),ax=-55,ay=0)
    fig.show()
    return

def zspbzs3():
    # 走势图3年
    fig = px.line(zspb3,x='DATES', y='PBMRQ', color='CODES')
    fig.update_layout(xaxis = dict(tickmode='linear',tick0 = 1,dtick = 120,),width=1200,height=600,title={'text': "指数PB走势3年",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                    title_font_size=35,font_size=15,title_font_color='red',xaxis_tickangle=-45,xaxis_title=None,yaxis_title=None)
    fig.update_yaxes(type='log')
    fig.show()
    return

In [ ]:
# 指数涨幅
zszfb()
zszf0()
zszf5()
zszf10()
zszf20()
zszf60()
zszf120()
zszf250()
#zszj0()
#zszj3()
#zszs20
#zszs3
zspbfw20()
zspbzs20()
zspbfw3()
zspbzs3()